In [ ]:
"""
 1.3 Tools and Autonomy
    Generally a no-framework approach to agentic AI engineering is best because nothing gets
    in your way and you can see/have control of the process end to end.

    That being said MCP is the key to providing your agents with tools and it is NOT a framework but a
    PROTOCOL.

    There are frameworks for agentic AI like LangGraph, but generally direct control is best.

1.3.1 Resources
We can give our agents more resources so that they can improve their expertise.
- A key way to do that is just to shove content in the prompt.
- There are also clever techinques like RAG (Retrieval Augmented Generation) to get smart relvant content.

1.3.2 Tools
Tools give agents POWER and AUTONOMY, it makes them truely effective as they can use these tools at it's discresion.
How this actually works is that we give a big list of all possible actions that it can do to the LLM, it responds in JSON
and in the JSON response it says what it wants to do. Then the tool (protocol) code, calls those actions in your code.

It's literally just responding in JSON and the protocol is able to call those things, it closes the 
loop by then calling the LLM back with the results.

SO.

> Prompt with actions 
    > JSON response 
        > Protocol calls python code 
            > Protocol calls same agent with the response.
"""

In [ ]:
# Imports!
from dotenv import load_dotenv
from openai import OpenAI
# Awesome PDF parser!
from pypdf import PdfReader
# Gradio can build beautiful FEs for data science.
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# Read the PDF into text so that we can feed into LLM
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [ ]:
# Get the summary as well as the PDF text
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
name = "Alessandro Alberga"

In [ ]:
"""
 You can specify the SYSTEM PROMPT. It is different to the USER PROMPT in the chat!

 the SYSTEM PROMPT `role:system` is used to "Set the scene" giving context for the task at hand, format
 and way it should respond.
"""
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
system_prompt

In [ ]:
# Chat is in the callback format that gradio needs.
def chat(message, history):
    # History ensures we are keeping the context of the conversation as it goes on!
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
"""
    Use GRADIO to interact with the LLM in a really nice UI.

    Cheezus this is sick!
"""
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

"""
    Class called BaseModel is used
"""
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
# You are meant to use a different model for evaluation but I am not really bothered
# for purposes of following along with these lectures
#import os
#gemini = OpenAI(
#    api_key=os.getenv("GOOGLE_API_KEY"), 
#    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
#)

In [ ]:
"""
    .parsed uses structured outputs,
    you give the schema class in response_format=Evaluation
"""
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
"""
    Rerun if the response was not acceptable.
"""
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
"""
    The full workflow!

    chat(message, history) allows you to hijack the chat and do the openai api calls
    directly.

    This allows you to execute code based on user input & additionally LLM output, Ausgezeichnet!
"""
def chat(message, history):
    # This is to trigger the retry and test!
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
# Wow now we have a chat interface for the workflow, mind blown.
gr.ChatInterface(chat, type="messages").launch()